### Build CNN Model for FashionMNIST Dataset :

In [66]:
!pip install torchvision

#### Necessary Imports :

In [67]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import transforms
from tqdm import tqdm

import warnings
warnings.filterwarnings(action="ignore")

#### Set device (CPU or GPU) :

In [68]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Define the CNN model :

In [69]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 32 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#### Define the transformation and load the FashionMNIST dataset :

In [70]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = FashionMNIST(root='./data', train=False, transform=transform)

#### Define dataloaders for training and testing :

In [71]:
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#### Initialize the model :

In [72]:
model = CNNModel().to(device)

#### Define the loss function and optimizer :

In [73]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### Training loop :

In [74]:
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Test Accuracy: {test_accuracy:.4f}%")

100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [00:36<00:00, 25.96it/s]


Epoch 1/10, Train Loss: 0.4792, Test Accuracy: 0.92%


100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [00:37<00:00, 25.19it/s]


Epoch 2/10, Train Loss: 0.3127, Test Accuracy: 0.92%


100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [00:43<00:00, 21.56it/s]


...

Epoch 8/10, Train Loss: 0.1612, Test Accuracy: 0.92%


100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [00:35<00:00, 26.08it/s]


Epoch 9/10, Train Loss: 0.1450, Test Accuracy: 0.92%


100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [00:34<00:00, 27.38it/s]

Epoch 10/10, Train Loss: 0.1317, Test Accuracy: 0.92%


#### Evaluation on test set :

In [75]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.9116


#### Save the model :

In [76]:
torch.save(model.state_dict(), 'fashion_mnist_cnn_model.pth')

### Model Improvement

#### Define the CNN model with improvements :

In [77]:
class ImprovedCNNModel(nn.Module):
    def __init__(self):
        super(ImprovedCNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 7 * 7, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.bn1(self.conv1(x))
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)
        x = self.bn2(self.conv2(x))
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = nn.functional.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

#### Set device (CPU or GPU) :

In [78]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Define the transformation and load the FashionMNIST dataset :

In [79]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = FashionMNIST(root='./data', train=False, transform=transform)

#### Define dataloaders for training and testing :

In [80]:
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#### Initialize the model :

In [81]:
improved_model = ImprovedCNNModel().to(device)

#### Define the loss function and optimizer :

In [82]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(improved_model.parameters(), lr=0.001)

#### Training loop :

In [84]:
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = improved_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Test Accuracy: {test_accuracy:.4f}%")

100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [01:29<00:00, 10.50it/s]


Epoch 1/10, Train Loss: 0.5669, Test Accuracy: 0.91%


100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [01:29<00:00, 10.49it/s]


Epoch 2/10, Train Loss: 0.3716, Test Accuracy: 0.91%


100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [01:32<00:00, 10.16it/s]


...

Epoch 8/10, Train Loss: 0.2149, Test Accuracy: 0.91%


100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [02:37<00:00,  5.94it/s]


Epoch 9/10, Train Loss: 0.2022, Test Accuracy: 0.91%


100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [05:31<00:00,  2.83it/s]

Epoch 10/10, Train Loss: 0.1927, Test Accuracy: 0.91%


#### Evaluation on test set :

In [85]:
improved_model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = improved_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.9176


#### Save the model :

In [86]:
torch.save(improved_model.state_dict(), 'improved_fashion_mnist_cnn_model.pth')

In [87]:
#The End.